In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import r2_score, root_mean_squared_error

# Importation des données nettoyées
donnees = pd.read_csv("data_nettoyer.csv")


In [2]:
# Séparation des variables explicatives et de la cible
X = donnees.drop(columns=['Frais'])
y = donnees['Frais']

# Séparation des colonnes numériques et catégoriques
colonnes_numeriques = ['Âge', 'IMC', 'Enfants']
colonnes_categoriques = ['Sexe', 'Fumeur', 'Région']


In [3]:
# Définition du ColumnTransformer
pretraitement = ColumnTransformer(
    transformers=[
        ("numérique", StandardScaler(), colonnes_numeriques),
        ("catégorique", OneHotEncoder(drop="first"), colonnes_categoriques)
    ],
    remainder="passthrough" # laisser intactes les colonnes non spécifiées
)


In [4]:

# Définition des modèles avec Pipeline
modeles = {
    "Régression Linéaire": Pipeline([
        ("pretraitement", pretraitement),
        ("modèle", LinearRegression())
    ]),
    "Lasso": Pipeline([
        ("pretraitement", pretraitement),
        ("poly", PolynomialFeatures(degree=2, include_bias=False)),
        ("re-scaler", StandardScaler()),
        ("modèle", Lasso(random_state=42))
    ]),
    "Ridge": Pipeline([
        ("pretraitement", pretraitement),
        ("modèle", Ridge(random_state=42))
    ]),
    "ElasticNet": Pipeline([
        ("pretraitement", pretraitement),
        ("modèle", ElasticNet(random_state=42))
    ])
}

In [5]:
# Division en données d'entraînement et de test (85% / 15%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['Fumeur']
)

In [6]:
# Évaluation des modèles
resultats = {}

for nom, pipeline in modeles.items():
    # Entraînement
    pipeline.fit(X_train, y_train)

    # Prédiction
    y_pred = pipeline.predict(X_test)

    # Évaluation
    r2 = r2_score(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)

    # Stockage des résultats
    resultats[nom] = {'R²': r2, 'RMSE': rmse}

    print(f"{nom} - R² : {r2:.4f}, RMSE : {rmse:.4f}")


Régression Linéaire - R² : 0.6816, RMSE : 6417.8223
Lasso - R² : 0.7921, RMSE : 5186.0549
Ridge - R² : 0.6819, RMSE : 6414.0960
ElasticNet - R² : 0.3400, RMSE : 9239.4006


In [11]:
modeles["Lasso"][-1].coef_

array([ 3.06826164e+03,  3.31367817e+02,  1.21244927e+03, -7.90282792e-02,
        9.84827408e+03, -9.97133522e+01, -3.80021521e-02, -2.88718813e+02,
        6.83942312e+02, -4.18788563e+01, -8.13638262e+01,  2.35773455e+02,
       -2.15151583e+01,  4.63207396e+01,  3.80792315e+02,  2.20224945e+02,
       -2.29257373e+02,  4.21387640e+01, -1.83572054e+01,  4.16859361e+03,
       -5.16246935e+01, -3.74439609e+02, -8.78691633e+00, -5.14669353e+01,
       -2.68385446e+02, -1.26225935e+02,  1.85680190e+02, -8.14446398e+01,
       -2.39386451e+02, -4.73160264e+02,  1.27160338e+02,  1.82016708e+01,
        3.62692803e+02,  9.27983528e+01,  0.00000000e+00, -1.19202260e+02,
       -5.96707927e+02,  6.96222333e+01, -9.08913953e-01,  0.00000000e+00,
        0.00000000e+00, -3.54433606e+02,  0.00000000e+00, -1.93690529e+02])

In [7]:
#Affichage des performances
resultats_df = pd.DataFrame(resultats).T
print("\nRésumé des performances :")
print(resultats_df)


Résumé des performances :
                           R²         RMSE
Régression Linéaire  0.681567  6417.822302
Lasso                0.792070  5186.054853
Ridge                0.681936  6414.096003
ElasticNet           0.340020  9239.400596


In [8]:

# GridSearchCV pour chaque modèle
parametres = {
    "Lasso": {"modèle__alpha": [0.1, 1.0, 10.0]},
    "Ridge": {"modèle__alpha": [0.1, 1.0, 10.0]},
    "ElasticNet": {"modèle__alpha": [0.1, 1.0, 10.0], "modèle__l1_ratio": [0.2, 0.5, 0.8]}
}

for nom, pipeline in modeles.items():
    if nom in parametres:
        grid = GridSearchCV(pipeline, parametres[nom], cv=5, scoring="neg_root_mean_squared_error")
        grid.fit(X_train, y_train)
        print(f"\nMeilleurs hyperparamètres pour {nom} :", grid.best_params_)
        print(f"Meilleur RMSE : {-grid.best_score_:.4f}")

/home/addeche/Documents/Projet_Simplon/Brief_Assur-Aimant/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.623e+07, tolerance: 1.315e+07
  model = cd_fast.enet_coordinate_descent(
/home/addeche/Documents/Projet_Simplon/Brief_Assur-Aimant/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.888e+07, tolerance: 1.329e+07
  model = cd_fast.enet_coordinate_descent(
/home/addeche/Documents/Projet_Simplon/Brief_Assur-Aimant/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You mi


Meilleurs hyperparamètres pour Lasso : {'modèle__alpha': 10.0}
Meilleur RMSE : 4813.2509

Meilleurs hyperparamètres pour Ridge : {'modèle__alpha': 0.1}
Meilleur RMSE : 5973.8916

Meilleurs hyperparamètres pour ElasticNet : {'modèle__alpha': 0.1, 'modèle__l1_ratio': 0.8}
Meilleur RMSE : 6064.1017


In [12]:
import pickle
with open("Lasso_pipeline.pkl", "wb") as file:
    pickle.dump(modeles["Lasso"], file)

In [13]:
with open("Lasso_pipeline.pkl", "rb") as file:
    loaded_model = pickle.load(file)


In [14]:
loaded_model.predict(X_train)

array([ 7369.98875212, 32914.40608101,  5408.57764916, ...,
       12459.67645125,  6529.08045826,  5726.41589166], shape=(1128,))